**Welcome to the GPU exercises of the topical CERN School of Computing Spring 2021 edition!**

Please follow the steps in this notebook. All underlying code can be found in the directories of the project. The files can be opened with an editor by clicking on the respective files. You can obtain syntax highlighting for .cu files by choosing `LANGUAGE` -> `C++` from the menu. We highly recommend that you inspect the source code of every exercise. 

Let us first explore the GPU available for you. `nvidia-smi` is a utility shipped with CUDA that monitors the processes running on a GPU and provides some information. It is often useful to check whether another user is using the same GPU.
Find out which GPU type you are provided with, whether a process is running and what the memory consumption is. Note that `nvidia-smi` also tells you the exact driver and CUDA version. This can be useful information if after a new installation / an update there is a mismatch between the CUDA and driver versions. 

In [ ]:
!nvidia-smi

Let us now compile and run small program, `device_properties/device_properties.cu`, which will give more detailed information about the GPU(s) available. This program does not do any calculations on the GPU, it simply queries information about it and shows you which function calls are available for that. Try t put the information you see into context with what you learned during the lectures!

Note: A similar program is available in the CUDA samples which are installed with every CUDA installation. You can find them in the directory `cuda-samples` of your CUDA installation directory (typically `/usr/local/cuda`). The scripts in `1_Utilities` can be useful.In particular, `1_Utilities/deviceQuery` gives similar (and more) information than the `device_properties`program we provide here. 

In [ ]:
!nvcc -arch=sm_70 -o device_properties/device_properties device_properties/device_properties.cu -run

We are now ready to look at a first actual CUDA program: `hello_world/hello_world.cu`. It prints a message from every invocated kernel to the terminal. The parameters to pass are the number of threads per block and the number of blocks in the grid. Try experimenting with different settings! In particular, try at least 64 or 96 threads per block and see if you can observe a pattern in the printout. What could it be due to?

In [4]:
!nvcc -arch=sm_70 -o hello_world/hello_world hello_world/hello_world.cu

In [ ]:
!./hello_world/hello_world 2 4